In [28]:
import torch
import csv
import pandas as pd
from sklearn.metrics import classification_report
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

Load Model & Tokenizer

In [2]:
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

Data Formatting

In [ ]:
def format_example(text, label=""):
    return f"""### Інструкція:
Класифікуй текст як один із класів:
Neutral або Sarcasm.

### Текст:
{text}

### Відповідь:
{label}"""

Evaluation Logic

In [23]:
def predict(text):
    prompt = (
        "Завдання: Визнач тональність тексту. "
        "Відповідь має бути одним словом: 'Sarcasm' або 'Neutral'.\n\n"
        f"Текст: {text}\n"
        "Категорія:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs, max_new_tokens=5, do_sample=False, pad_token_id=tokenizer.eos_token_id
    )

    input_length = inputs.input_ids.shape[1]
    prediction_tokens = outputs[0][input_length:]
    decoded = (
        tokenizer.decode(prediction_tokens, skip_special_tokens=True).strip().lower()
    )

    # Map the word response to a binary integer
    if "sarcasm" in decoded:
        return 1
    return 0  # Default to Neutral (0)

Result logging

In [ ]:
def log_result(stage, text, gold, pred, path="results.csv"):
    header = ["stage", "text", "gold", "pred"]

    # Force gold and pred to integers to ensure consistency in the CSV
    row = [stage, text, int(gold), int(pred)]

    file_exists = os.path.isfile(path)

    with open(path, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(row)

Zero-shot baseline

In [ ]:
test_texts = [
    ("Його вигнали з роботи тому що він розмовляв російською.", 0),
    ("Сьогодні так класно погуляли з друзями, і погода була супер.", 0),
    ("Розумні думки намагаються тебе наздогнати, але ти бистріший.", 1),
    (
        "Так, звичайно, повтори мені це ще 5 разів, а то я перший раз не почув)",
        1,
    ),
]

for text, gold in test_texts:
    pred = predict(text)
    log_result("zero-shot-ukrainian", text, gold, pred)
    print(f"Текст: {text}\nExpected: {gold} | Predicted: {pred}\n" + "-" * 30)

c:\hate_speech_ukranian_llm\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Текст: Його вигнали з роботи тому що він розмовляв російською.
Expected: 0 | Predicted: 0
------------------------------
Текст: Сьогодні так класно погуляли з друзями, і погода була супер.
Expected: 0 | Predicted: 0
------------------------------
Текст: Розумні думки намагаються тебе наздогнати, але ти бистріший.
Expected: 1 | Predicted: 0
------------------------------
Текст: Так, звичайно, повтори мені це ще 5 разів, а то я перший раз не почув)
Expected: 1 | Predicted: 0
------------------------------


Compute Accuracy

In [29]:
df = pd.read_csv("results.csv")

# Print overall accuracy for each stage
acc = df.groupby("stage").apply(
    lambda x: (x["gold"] == x["pred"]).mean(), include_groups=False
)
print("--- Accuracy ---")
print(acc)

# Detailed report (Precision, Recall, F1)
# Use target_names to keep it readable
for stage in df["stage"].unique():
    subset = df[df["stage"] == stage]
    print(f"\nReport for {stage}:")
    print(
        classification_report(
            subset["gold"], subset["pred"], target_names=["Neutral", "Sarcasm"]
        )
    )

--- Accuracy ---
stage
zero-shot    0.5
dtype: float64

Report for zero-shot:


ValueError: Number of classes, 4, does not match size of target_names, 2. Try specifying the labels parameter

LoRA Setup

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

Training Arguments & Trainer

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,
    output_dir="./results",
    save_strategy="epoch",
    evaluation_strategy="no",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    max_seq_length=512,
    args=training_args,
)

Start Training

In [ ]:
trainer.train()